# Understanding the Agent Loop

**Deep Dive into How Agents Think and Act**

---

Welcome to an in-depth exploration of the **Strands Agent Loop**! This notebook reveals the inner workings of how agents process information, make decisions, and generate responses. By the end of this 10-minute tutorial, you'll understand the elegant architecture that powers intelligent agent behavior.

### 🎯 What You'll Learn

In this technical deep-dive, you will:
- Understand the core components of the agent loop
- Trace how messages flow through the system
- See how agents decide when to use tools
- Learn about recursive reasoning and multi-step workflows
- Debug and monitor agent decision-making
- Build intuition for creating more sophisticated agents

### 🔄 What is the Agent Loop?

The **Agent Loop** is the heart of every Strands agent. It's a sophisticated cycle that:
1. **Receives** user input
2. **Processes** it with an AI model
3. **Decides** on actions (including tool use)
4. **Executes** those actions
5. **Reasons** about results
6. **Generates** responses

This loop can iterate multiple times in a single interaction, enabling complex reasoning!

## 📦 Step 1: Setup and Imports

### Installing Dependencies
We'll need the core Strands SDK and some example tools to demonstrate the agent loop in action.

In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools -q

# Import necessary modules
import json
import time
import logging
from strands import Agent, tool
from strands_tools import calculator, current_time

print("✅ Setup complete! Ready to explore the agent loop.")

## 🔍 Step 2: Enabling Debug Logging

### See What's Happening Under the Hood
To understand the agent loop, we need to see what's happening internally. Let's enable debug logging to trace the agent's thought process.

In [ ]:
# 🚀 EASIEST OPTION: Paste your Bedrock API Key here
# Get your API key from: https://console.aws.amazon.com/bedrock/ -> API keys
API_KEY = ""  # Paste your API key between the quotes

# Configuration constants (you can modify these if needed)
REGION_NAME = "us-west-2"
AWS_PROFILE = "default"

# Set up authentication using our utility function
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / "src"))  # Add the src directory to path for imports
from auth_utils import setup_bedrock_auth, display_auth_status

auth_status = setup_bedrock_auth(
    api_key=API_KEY,
    region_name=REGION_NAME,
    aws_profile=AWS_PROFILE
)

# Display the authentication status
display_auth_status(auth_status)

# Configure logging to see agent internals
logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()],
    level=logging.INFO
)

print("\n🔍 Debug logging enabled!")
print("   You'll now see detailed information about the agent's internal processes.")

## 🎭 Step 3: The Basic Agent Loop

### Single-Turn Interaction
Let's start with the simplest case: an agent without tools responding to a single message. This demonstrates the basic flow of the agent loop.

In [ ]:
# Create a simple agent without tools
simple_agent = Agent(
    model=auth_status.strands_bedrock_model,
    system_prompt="You are a helpful assistant. Be concise but informative."
)

print("🎭 BASIC AGENT LOOP DEMONSTRATION")
print("=" * 50)
print("\n📍 Step 1: User sends a message")
print("📍 Step 2: Agent processes with AI model")
print("📍 Step 3: Agent returns response\n")

# Send a simple message
start_time = time.time()
response = simple_agent("What is the capital of France?")
end_time = time.time()

print(f"\n🤖 Agent Response: {response}")
print(f"⏱️  Processing Time: {end_time - start_time:.2f} seconds")

# Show the conversation history
print("\n📜 Conversation History:")
for i, msg in enumerate(simple_agent.messages):
    role = msg.get('role', 'unknown')
    content = msg.get('content', [])[0].get('text', '') if msg.get('content') else ''
    print(f"   Message {i+1} ({role}): {content[:50]}...")

## 🔧 Step 4: The Tool-Using Agent Loop

### Multi-Step Reasoning
Now let's see how the agent loop handles tool usage. When an agent has tools available, the loop becomes more complex:

1. **User Input** → 2. **Model Processing** → 3. **Tool Request** → 4. **Tool Execution** → 5. **Result Processing** → 6. **Final Response**

In [ ]:
# Create an agent with tools
tool_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[calculator, current_time],
    system_prompt="You are a helpful assistant with access to tools. Use them when needed."
)

print("🔧 TOOL-USING AGENT LOOP DEMONSTRATION")
print("=" * 50)
print("\n📍 Watch how the agent loop handles tool usage:\n")

# Send a message that requires tool use
start_time = time.time()
response = tool_agent("What is 25 * 48? Also, what time is it?")
end_time = time.time()

print(f"\n🤖 Final Response: {response}")
print(f"⏱️  Total Processing Time: {end_time - start_time:.2f} seconds")

# Analyze the conversation history
print("\n📊 Agent Loop Analysis:")
print("=" * 50)
for i, msg in enumerate(tool_agent.messages):
    role = msg.get('role', 'unknown')
    content = msg.get('content', [])
    
    print(f"\n🔄 Loop Step {i+1} - Role: {role}")
    
    for item in content:
        if 'text' in item:
            print(f"   💬 Text: {item['text'][:100]}...")
        elif 'toolUse' in item:
            tool_use = item['toolUse']
            print(f"   🔧 Tool Request: {tool_use.get('name')}")
            print(f"      Input: {tool_use.get('input')}")
        elif 'toolResult' in item:
            tool_result = item['toolResult']
            print(f"   ✅ Tool Result: {tool_result.get('content', [{}])[0].get('text', 'N/A')}")

## 🌀 Step 5: Recursive Agent Loops

### Complex Multi-Step Reasoning
The agent loop can recursively call itself when multiple tool uses are needed. Let's create a scenario that demonstrates this recursive behavior.

In [ ]:
# Create custom tools to demonstrate recursive loops
@tool
def get_word_count(text: str) -> int:
    """Count the number of words in a text."""
    return len(text.split())

@tool
def get_character_count(text: str) -> int:
    """Count the number of characters in a text."""
    return len(text)

@tool
def calculate_reading_time(word_count: int) -> float:
    """Calculate estimated reading time in minutes (200 words per minute)."""
    return round(word_count / 200, 2)

# Create an agent with these tools
recursive_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[get_word_count, get_character_count, calculate_reading_time],
    system_prompt="You are a text analysis assistant. Analyze text thoroughly using available tools."
)

print("🌀 RECURSIVE AGENT LOOP DEMONSTRATION")
print("=" * 50)
print("\n📍 Watch how the agent makes multiple tool calls:\n")

# Text to analyze
sample_text = """The agent loop is a fascinating concept in AI. It allows agents to reason 
step by step, use tools when needed, and build complex responses through iterative processing."""

# Send the analysis request
response = recursive_agent(f"Analyze this text and give me all statistics: '{sample_text}'")

print(f"\n🤖 Final Analysis: {response}")

# Count the number of tool uses
tool_uses = 0
for msg in recursive_agent.messages:
    for item in msg.get('content', []):
        if 'toolUse' in item:
            tool_uses += 1

print(f"\n📊 Loop Statistics:")
print(f"   Total messages: {len(recursive_agent.messages)}")
print(f"   Tool uses: {tool_uses}")
print(f"   Loop iterations: {(len(recursive_agent.messages) - 1) // 2}")

## 🎯 Step 6: Tracing Agent Decisions

### Understanding Why Agents Make Choices
Let's create a custom callback handler to trace exactly what happens at each step of the agent loop. This gives us deep insights into the decision-making process.

In [ ]:
# Create a detailed callback handler
class AgentLoopTracer:
    def __init__(self):
        self.events = []
        self.current_step = 0
    
    def __call__(self, **kwargs):
        self.current_step += 1
        event = {
            'step': self.current_step,
            'timestamp': time.time(),
            'type': 'unknown'
        }
        
        if 'data' in kwargs:
            event['type'] = 'text_generation'
            event['data'] = kwargs['data'][:50] + '...' if len(kwargs['data']) > 50 else kwargs['data']
        elif 'current_tool_use' in kwargs:
            event['type'] = 'tool_decision'
            event['tool'] = kwargs['current_tool_use'].get('name')
        elif 'event_type' in kwargs:
            event['type'] = kwargs['event_type']
            
        self.events.append(event)
    
    def print_trace(self):
        print("\n🔍 AGENT LOOP TRACE:")
        print("=" * 50)
        start_time = self.events[0]['timestamp'] if self.events else 0
        
        for event in self.events:
            elapsed = event['timestamp'] - start_time
            print(f"\n⏱️  +{elapsed:.3f}s - Step {event['step']}")
            print(f"   Type: {event['type']}")
            
            if event['type'] == 'text_generation':
                print(f"   Generated: {event.get('data', 'N/A')}")
            elif event['type'] == 'tool_decision':
                print(f"   Tool: {event.get('tool', 'N/A')}")

# Create tracer and agent
tracer = AgentLoopTracer()
traced_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[calculator, current_time],
    callback_handler=tracer
)

print("🎯 TRACING AGENT DECISIONS")
print("=" * 50)

# Execute a complex request
response = traced_agent(
    "Calculate 156 * 89, then tell me what time it is. "
    "Finally, calculate how many seconds are in the result of the first calculation."
)

# Print the trace
tracer.print_trace()

print(f"\n\n🤖 Final Response: {response}")

## 🔄 Step 7: Parallel Tool Execution

### Optimizing the Agent Loop
Strands agents can execute multiple tools in parallel when they're independent. This optimization in the agent loop significantly improves performance.

In [ ]:
# Create tools that simulate longer operations
@tool
def slow_calculation(x: int, y: int) -> int:
    """Perform a slow calculation (simulated)."""
    time.sleep(1)  # Simulate processing time
    return x + y

@tool
def slow_multiplication(x: int, y: int) -> int:
    """Perform a slow multiplication (simulated)."""
    time.sleep(1)  # Simulate processing time
    return x * y

# Create agents with different parallel settings
sequential_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[slow_calculation, slow_multiplication],
    max_parallel_tools=1  # Sequential execution
)

parallel_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[slow_calculation, slow_multiplication],
    max_parallel_tools=4  # Parallel execution
)

print("🔄 PARALLEL VS SEQUENTIAL EXECUTION")
print("=" * 50)

# Test sequential execution
print("\n1️⃣ Sequential Execution (max_parallel_tools=1):")
start_time = time.time()
seq_response = sequential_agent(
    "Calculate 10 + 20 and 5 * 6. These are independent calculations."
)
seq_time = time.time() - start_time
print(f"   Response: {seq_response}")
print(f"   ⏱️  Time: {seq_time:.2f} seconds")

# Test parallel execution
print("\n2️⃣ Parallel Execution (max_parallel_tools=4):")
start_time = time.time()
par_response = parallel_agent(
    "Calculate 10 + 20 and 5 * 6. These are independent calculations."
)
par_time = time.time() - start_time
print(f"   Response: {par_response}")
print(f"   ⏱️  Time: {par_time:.2f} seconds")

print(f"\n⚡ Performance Improvement: {seq_time/par_time:.1f}x faster with parallel execution!")

## 🧠 Step 8: Understanding Context Windows

### How Agents Manage Long Conversations
The agent loop includes conversation management to handle context window limits. Let's explore how this works.

In [ ]:
# Create an agent with a small conversation window
from strands.agent.conversation_manager import SlidingWindowConversationManager

# Create a conversation manager with a small window
small_window_manager = SlidingWindowConversationManager(
    window_size=3  # Only keep 3 message pairs
)

windowed_agent = Agent(
    model=auth_status.strands_bedrock_model,
    conversation_manager=small_window_manager,
    system_prompt="You are a helpful assistant."
)

print("🧠 CONTEXT WINDOW MANAGEMENT")
print("=" * 50)
print("\n📍 Agent configured with window_size=3 (keeps only recent messages)\n")

# Have a longer conversation
topics = [
    "Hi! Let's talk about planets.",
    "Tell me about Mars.",
    "What about Jupiter?",
    "How about Saturn?",
    "What was the first planet we discussed?"
]

for i, topic in enumerate(topics):
    print(f"\n👤 Turn {i+1}: {topic}")
    response = windowed_agent(topic)
    print(f"🤖 Agent: {response}")
    
    # Show current conversation window
    print(f"\n📊 Current Window Size: {len(windowed_agent.messages)} messages")
    if i == len(topics) - 1:
        print("\n⚠️  Notice: The agent may not remember Mars because it's outside the window!")

## 🎨 Step 9: Custom Agent Loop Components

### Building Your Own Loop Extensions
The agent loop is extensible. Let's create a custom component that adds timing information to each loop iteration.

In [ ]:
# Create a custom tool that tracks its own execution
class TimedTool:
    def __init__(self, name, func):
        self.name = name
        self.func = func
        self.execution_times = []
    
    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self.func(*args, **kwargs)
        duration = time.time() - start
        self.execution_times.append(duration)
        return f"{result} (took {duration:.3f}s)"

# Create timed versions of tools
@tool
def fibonacci(n: int) -> int:
    """Calculate the nth Fibonacci number."""
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

@tool
def factorial(n: int) -> int:
    """Calculate the factorial of n."""
    if n <= 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Create agent with custom tools
math_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[fibonacci, factorial],
    system_prompt="You are a mathematics assistant. Show your calculations."
)

print("🎨 CUSTOM AGENT LOOP COMPONENTS")
print("=" * 50)

# Complex mathematical request
response = math_agent(
    "Calculate the 10th Fibonacci number and the factorial of 7. "
    "Then explain what these sequences represent."
)

print(f"\n🤖 Response: {response}")

# Analyze the loop execution
print("\n📊 Agent Loop Execution Analysis:")
tool_count = sum(1 for msg in math_agent.messages 
                 for item in msg.get('content', []) 
                 if 'toolUse' in item)
print(f"   Tool invocations: {tool_count}")
print(f"   Total loop iterations: {len(math_agent.messages)}")
print(f"   Final response length: {len(str(response))} characters")

## 🔬 Step 10: Advanced Loop Debugging

### Inspecting Every Detail
Let's create a comprehensive debugging setup that shows every aspect of the agent loop in action.

In [ ]:
# Create a comprehensive debugging callback
class DebugCallback:
    def __init__(self):
        self.events = []
        
    def __call__(self, **kwargs):
        event = {
            'timestamp': time.time(),
            'kwargs': kwargs
        }
        self.events.append(event)
        
        # Real-time output for specific events
        if 'event_type' in kwargs:
            if kwargs['event_type'] == 'tool_use_delta':
                print(f"🔧 Tool Delta: {kwargs.get('current_tool_use', {}).get('name')}")
            elif kwargs['event_type'] == 'text_delta':
                print("📝 Text Generation in progress...")
        elif 'data' in kwargs:
            # Streaming text
            print(kwargs['data'], end='', flush=True)

# Create a debug agent
debug_callback = DebugCallback()
debug_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[calculator],
    callback_handler=debug_callback,
    system_prompt="You are a helpful assistant. Think step by step."
)

print("🔬 ADVANCED AGENT LOOP DEBUGGING")
print("=" * 50)
print("\n📍 Watch the real-time agent loop execution:\n")

# Execute with debugging
result = debug_agent(
    "If I have 15 apples and buy 3 times as many oranges, "
    "how many fruits do I have in total?"
)

# Analyze the debug events
print("\n\n📊 Debug Event Summary:")
print("=" * 50)

event_types = {}
for event in debug_callback.events:
    event_type = event['kwargs'].get('event_type', 'text_stream')
    event_types[event_type] = event_types.get(event_type, 0) + 1

for event_type, count in event_types.items():
    print(f"   {event_type}: {count} events")

print(f"\n   Total events captured: {len(debug_callback.events)}")
print(f"   Total execution time: {debug_callback.events[-1]['timestamp'] - debug_callback.events[0]['timestamp']:.2f}s")

## 🎯 Summary: The Agent Loop Architecture

### 🏗️ Key Components We've Explored

1. **Message Processing Pipeline**
   - User input → Model processing → Response generation
   - Automatic conversation history management
   - Context window handling

2. **Tool Integration Flow**
   - Model decides when to use tools
   - Tool execution (sequential or parallel)
   - Result processing and reasoning
   - Recursive loops for complex tasks

3. **Decision Making Process**
   - AI model analyzes user intent
   - Determines appropriate tools to use
   - Chains multiple operations together
   - Synthesizes final response

4. **Performance Optimizations**
   - Parallel tool execution
   - Efficient message handling
   - Smart context management

### 🚀 What You've Learned

You now understand:
- ✅ How agents process information step-by-step
- ✅ The recursive nature of complex reasoning
- ✅ Tool execution patterns and optimization
- ✅ Debugging and monitoring techniques
- ✅ Context window management strategies

### 💡 Key Insights

1. **The Loop is Recursive**: Agents can call themselves multiple times to complete complex tasks
2. **Tools Enable Intelligence**: The combination of LLMs and tools creates powerful capabilities
3. **Parallelism Matters**: Independent operations can run simultaneously for better performance
4. **Observability is Crucial**: Understanding the loop helps debug and improve agents

### 🎯 Next Steps

With this deep understanding of the agent loop, you're ready to:
- Build more sophisticated agents with complex workflows
- Debug agent behavior effectively
- Optimize performance for production use
- Create custom extensions to the agent loop

### 📚 Further Reading

- [Agent Loop Documentation](https://strandsagents.com/0.1.x/user-guide/concepts/agents/agent-loop/)
- [Tool Development Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/)
- [Streaming and Callbacks](https://strandsagents.com/0.1.x/user-guide/concepts/streaming/)

### 🎊 Congratulations!

You've mastered the inner workings of the Strands agent loop! This knowledge is fundamental for building advanced AI systems. In the next videos, we'll explore streaming responses, state management, and multi-agent systems.

Remember: The agent loop is the engine that powers intelligent behavior. Understanding it deeply makes you a more effective AI developer! 🚀